In [1]:
import pandas as pd

file_path = '../../real_data/trace/W2K8.TPCC.10-19-2007.12-51-PM.trace-filtered-target.txt'

In [2]:
#  read the file
trace = pd.read_csv(file_path, sep=' ')
trace.columns = ['ByteOffset', 'IOSize']
print(len(trace))
print(trace.head())

5899528
   ByteOffset  IOSize
0  3201773568    8192
1  3155169280    4096
2  3155156992    4096
3  3161305088    8192
4  3154132992    4096


In [3]:
# declare dataframe variable to store converted trace
converted_trace = pd.DataFrame(columns=['ByteOffset', 'IOSize'])
counter = 0
# READ each line of the trace
for index, row in trace.iterrows():
    converted_list = []
    # IOSize to sector number and save it to the new trace
    sector_number = row['IOSize'] // 512
    # add a sector in replace of io size
    converted_list.append(row['ByteOffset'])
    converted_list.append(sector_number)
    converted_trace.loc[counter] = converted_list
    counter += 1
    if counter % 100000 == 0:
        print(counter)
    
print(converted_trace.head())

    


KeyboardInterrupt: 

In [29]:
# Save the converted trace
converted_trace.to_csv('../../real_data/trace/W2K8.TPCC.10-19-2007.12-51-PM.trace-filtered-target-converted.txt', sep=' ', index=False)

In [ ]:
# if the converted_trace i/o size is greater than 8 then we need to split it into multiple i/o
# if the converted_trace i/o size is less than 8 then we need to merge it into one i/o
# if the converted_trace i/o size is equal to 8 then we don't need to do anything

# declare dataframe variable to store split trace
split_trace = pd.DataFrame(columns=['ByteOffset', 'IOSize'])
counter = 0
# READ each line of the trace

for index, row in converted_trace.iterrows():
    # if i/o size is greater than 8 then split it into multiple i/o
    if row['IOSize'] > 8:
        # calculate the number of i/o
        number_of_io = row['IOSize'] // 8
        # calculate the remaining i/o
        remaining_io = row['IOSize'] % 8
        # split the i/o into 8
        for i in range(number_of_io):
            split_trace.loc[counter] = [row['ByteOffset']+(i*8), 8]
            counter += 1
        # if there is any remaining i/o then add it to the split trace
        if remaining_io != 0:
            split_trace.loc[counter] = [row['ByteOffset']+(number_of_io*8), remaining_io]
    # if i/o size is less than 8 then merge it into one i/o
    elif row['IOSize'] < 8:
        # calculate the remaining i/o
        remaining_io = row['IOSize']
        # merge the i/o into 8
        split_trace.loc[counter] = [row['ByteOffset'], remaining_io]
        counter += 1
    # if i/o size is equal to 8 then don't need to do anything
    else:
        split_trace.loc[counter] = [row['ByteOffset'], 8]
        counter += 1
    if counter > 2000000:
        break
    if counter % 10000 == 0:
        print(counter)

# save the split trace
split_trace.to_csv('../../real_data/trace/W2K8.TPCC.10-19-2007.12-51-PM.trace-filtered-target-converted-split.txt', sep=' ', index=False)

10000
70000
120000
180000
200000
220000
260000
280000
320000
340000
350000
370000
380000
410000
420000
500000
510000
520000
530000
540000
560000
570000
580000
640000
670000
680000
690000
730000
750000
860000
930000
940000
950000
1010000
1020000
1080000
1110000
1120000
1160000
1170000
1180000
1340000
1350000
1370000
1480000
1500000
1510000
1520000
1540000
1590000
1610000
1640000
1650000
1670000
1680000
1700000
1760000
1780000
1800000
1830000
1860000
1890000
1900000
1910000
1920000
1940000
1950000
1970000
1990000


In [36]:
print(split_trace.head())

   ByteOffset  IOSize
0  3201773568       8
1  3201773576       8
2  3155169280       8
3  3155156992       8
4  3161305088       8


# YCSB RocksDB SSD traces

In [2]:
import pandas as pd

file_path = '../../real_data/trace/ssdtrace-00'

In [ ]:
import pandas as pd

trace = pd.read_csv(file_path, delim_whitespace=True, header=None, usecols=[6, 7, 9])
trace.columns = ['Type', 'ByteOffset', 'IOSize']
write_trace = trace[(trace['Type'] == 'W')]


In [ ]:
# if the converted_trace i/o size is greater than 8 then we need to split it into multiple i/o
# if the converted_trace i/o size is less than 8 then we need to merge it into one i/o
# if the converted_trace i/o size is equal to 8 then we don't need to do anything

# declare dataframe variable to store split trace
split_trace = pd.DataFrame(columns=['ByteOffset', 'IOSize'])
counter = 0
# READ each line of the trace

for index, row in write_trace.iterrows():
    # if i/o size is greater than 8 then split it into multiple i/o
    # check row['ByteOffset'] is integer or not
    # print(row['ByteOffset'])
    # check byteoffset is integer or not
    try:
        int(row['ByteOffset'])
        if row['IOSize'] > 8:
            # calculate the number of i/o
            number_of_io = int(row['IOSize'] // 8)
            # calculate the remaining i/o
            remaining_io = row['IOSize'] % 8
            # split the i/o into 8
            for i in range(number_of_io):
                split_trace.loc[counter] = [int(row['ByteOffset'])+(i*8), 8]
                counter += 1
            # if there is any remaining i/o then add it to the split trace
            if remaining_io != 0:
                split_trace.loc[counter] = [int(row['ByteOffset'])+(number_of_io*8), remaining_io]
        # if i/o size is less than 8 then merge it into one i/o
        elif row['IOSize'] < 8:
            # calculate the remaining i/o
            remaining_io = row['IOSize']
            # merge the i/o into 8
            split_trace.loc[counter] = [int(row['ByteOffset']), remaining_io]
            counter += 1
        # if i/o size is equal to 8 then don't need to do anything
        else:
            split_trace.loc[counter] = [int(row['ByteOffset']), 8]
            counter += 1
        if counter > 2000000:
            break
        if index % 100 == 0:
            print(index)
    except:
        pass

# save the split trace
split_trace.to_csv('../../real_data/trace/ssdtrace-00-split.txt', sep=' ', index=False)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2100
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3700
3800
3900
4000
4200
4300
4400
4600
4700
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7200
7300
7400
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10100
10200
10300
10500
10600
10700
10800
10900
11000
11100
11300
11400
11500
11700
11800
11900
12100
12200
12400
12500
12600
12700
12800
13000
13200
13300
13400
13500
13600
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15100
15200
15300
15400
15500
15700
15800
15900
16000
16100
16300
16400
16600
16700
16800
16900
17000
17300
17600
17700
17800
18000
18100
18200
18300
18400
18500
18600
18700
18800
18900
19000
19100
19200
19300
19400
19500
19600
19700
19800
19900
20100
20200
20300
20400
20500
20600
20700
20800
20900
21100
21200
21300

In [6]:
split_trace.head(20)

,ByteOffset,IOSize
0,282624,8
1,282624,8
2,282624,8
3,282624,8
4,282624,8
5,286720,8
6,286728,8
7,286736,8
8,286744,8
9,286752,8


In [8]:
trace.head(200)

,Type,LBA,Size
0,R,282624,8.0
1,R,282624,8.0
2,N,[java],NaN
3,RS,282624,8.0
4,RS,282624,8.0
...,...,...,...
195,R,805698992,256.0
196,N,[java],NaN
197,RS,805698992,256.0
198,RS,805698992,256.0


In [30]:
for i in range(3):
    print(i)

0
1
2
